In [ ]:
import json
import pandas as pd
import ast
import csv
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import get_scheduler
from tqdm import tqdm

In [ ]:
%cd /content

/content


In [ ]:
!rm -r /content/ML-Project---bioBERT

rm: cannot remove '/content/ML-Project---bioBERT': No such file or directory


In [ ]:
!git clone https://github.com/excellencior/ML-Project---bioBERT.git

Cloning into 'ML-Project---bioBERT'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 166 (delta 55), reused 130 (delta 31), pack-reused 0 (from 0)
Receiving objects: 100% (166/166), 34.23 MiB | 8.48 MiB/s, done.
Resolving deltas: 100% (55/55), done.


# Dataset Processing

In [ ]:
# Create the dataset directory
!mkdir -p dataset

# Download files into the dataset directory
!wget -O dataset/train.csv https://huggingface.co/datasets/aai530-group6/ddxplus/resolve/main/train.csv
!wget -O dataset/validate.csv https://huggingface.co/datasets/aai530-group6/ddxplus/resolve/main/validate.csv
!wget -O dataset/test.csv https://huggingface.co/datasets/aai530-group6/ddxplus/resolve/main/test.csv


--2024-12-17 12:42:17--  https://huggingface.co/datasets/aai530-group6/ddxplus/resolve/main/train.csv
Resolving huggingface.co (huggingface.co)... 3.165.160.12, 3.165.160.61, 3.165.160.11, ...
Connecting to huggingface.co (huggingface.co)|3.165.160.12|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/3d/bf/3dbf1ec86c02d693ed39ac3d592b3fd0ea3c9e85890918306b3db431c821f0f7/93933e9a9a7a00d618a931deda7767485af299fe4635bf189a7c63b510e8b172?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27train.csv%3B+filename%3D%22train.csv%22%3B&response-content-type=text%2Fcsv&Expires=1734698537&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNDY5ODUzN319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzNkL2JmLzNkYmYxZWM4NmMwMmQ2OTNlZDM5YWMzZDU5MmIzZmQwZWEzYzllODU4OTA5MTgzMDZiM2RiNDMxYzgyMWYwZjcvOTM5MzNlOWE5YTdhMDBkNjE4YTkzMWRlZGE3NzY3NDg1YWYyOTlmZTQ2MzViZjE4OWE3YzYzYjUxMGU4YjE3

In [ ]:
import pandas as pd
import json

def parse_evidences2(evidences, json_data):
    """
    Parse evidences into meaningful text using JSON mappings and combine repeated questions' answers with &.
    """
    parsed_antecedents = {}
    parsed_symptoms = {}
    for evidence in eval(evidences):  # Convert string list to actual list
        if "_@_" in evidence:
            code, value = evidence.split("_@_")
            question = json_data.get(code, {}).get('question_en', 'Unknown question')
            value_meaning = json_data.get(code, {}).get('value_meaning', {}).get(value, {}).get('en', value)
            is_antecedent = json_data.get(code, {}).get('is_antecedent', False)
            target_dict = parsed_antecedents if is_antecedent else parsed_symptoms
            if question in target_dict:
                target_dict[question] += f" & {value_meaning}"
            else:
                target_dict[question] = value_meaning
        else:
            question = json_data.get(evidence, {}).get('question_en', 'Unknown question')
            is_antecedent = json_data.get(evidence, {}).get('is_antecedent', False)
            target_dict = parsed_antecedents if is_antecedent else parsed_symptoms
            if question in target_dict:
                target_dict[question] += " & Y"
            else:
                target_dict[question] = "Y"

    antecedents = [f"{q} - {a}" for q, a in parsed_antecedents.items()]
    symptoms = [f"{q} - {a}" for q, a in parsed_symptoms.items()]
    return antecedents, symptoms

def transform_data(csv_path, json_path, output_path):
    """
    Transforms the CSV and JSON data into BioBERT-friendly format.
    """
    # Load JSON and CSV files
    with open(json_path, 'r') as file:
        json_data = json.load(file)

    csv_data = pd.read_csv(csv_path)

    # Process each row in the CSV
    formatted_data = []

    for _, row in csv_data.iterrows():
        patient_data = {
            "Age": row['AGE'],
            "Sex": row['SEX'],
            "Antecedents": [],
            "Symptoms": [],
            "Differential Diagnosis": [],
        }

        # Parse evidences
        antecedents, symptoms = parse_evidences2(row['EVIDENCES'], json_data)
        patient_data["Antecedents"].extend(antecedents)
        patient_data["Symptoms"].extend(symptoms)

        # Parse differential diagnosis (exclude probabilities)
        diagnoses = [diag[0] for diag in eval(row['DIFFERENTIAL_DIAGNOSIS'])]
        patient_data["Differential Diagnosis"] = diagnoses

        formatted_data.append(patient_data)

    # Save the formatted data to a JSON file
    with open(output_path, 'w') as output_file:
        json.dump(formatted_data, output_file, indent=4)

task = "val"
sample_size = 20000
data = pd.read_csv(f'/content/ML-Project---bioBERT/dataset/{task}.csv')
data.sample(n=sample_size).to_csv(f'/content/ML-Project---bioBERT/dataset_processed/csv/{task}_sample{sample_size}.csv', index=False)

# Paths to input files and output location
csv_path = f'/content/ML-Project---bioBERT/dataset_processed/csv/{task}_sample{sample_size}.csv'  # Replace with the actual CSV file path
json_path = '/content/ML-Project---bioBERT/release_evidences_cleaned.json'  # Replace with the actual JSON file path
output_path = f'/content/ML-Project---bioBERT/dataset_processed/json/{task}_sample{sample_size}.json'  # Desired output file name

# Transform the data
transform_data(csv_path, json_path, output_path)

# Train-Test bioBERT

In [ ]:
!pwd

/content


In [ ]:
%cd '/content/ML-Project---bioBERT'

/content/ML-Project---bioBERT


In [ ]:
!python bioBERT_finetune.py

2024-12-17 13:41:11.166635: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-17 13:41:11.197352: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-17 13:41:11.206718: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-17 13:41:12.752559: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
--- Train_Set ---
Dataset Encoding:
Number of samples: 5000
Number of unique labels: 49

--- Validation_Set ---
Dataset Encoding:
Number of samples: 2500
Number of unique labels: 49

Testing: 100% 313/313 [01:32<00:00,  3.37it/s]

--- Evalua

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r '/content/drive/MyDrive/20k Samples' '.'